In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"  # Choose which GPUs by checking current use with nvidia-smi
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import metrics
import time
import numpy as np

# Check CUDA functionality, restart kernel to change GPUs
gpus = tf.config.list_physical_devices('GPU')
print("*************************************")
print(gpus)
print("*************************************")

if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    
# Define function to preprocess images as required by ResNet
def preprocess(images, labels):
    return tf.keras.applications.resnet_v2.preprocess_input(images), labels


# setup train, validation, and test folders
traindir = '/mnt/d/datasets/INBREAST/split/train'
valdir = '/mnt/d/datasets/INBREAST/split/validation'
testdir = '/mnt/d/datasets/INBREAST/split/test'
dirName = '2_classes'

buffersize = 2
# im_dim = 512
im_dim_x = 224
im_dim_y = 224
batchSizeIntInitial = 10
batchSizeInt = 6

train = tf.keras.preprocessing.image_dataset_from_directory(
    traindir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)
val = tf.keras.preprocessing.image_dataset_from_directory(
    valdir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)
test = tf.keras.preprocessing.image_dataset_from_directory(
    testdir, image_size=(im_dim_x, im_dim_y), batch_size=batchSizeInt)

test_ds = test.map(preprocess)
train_ds = train.map(preprocess)
val_ds = val.map(preprocess)
train_ds = train_ds.prefetch(buffer_size=buffersize)
val_ds = val_ds.prefetch(buffer_size=buffersize)

## set up hyperparameters, such as epochs, learning rates, cutoffs.
epochs = 30
lr = 0.004
cutoff = 0.5
start_time = time.time()
mirrored_strategy = tf.distribute.MirroredStrategy()

with mirrored_strategy.scope():  # the entire model needs to be compiled within the scope of the distribution strategy
    # cb1 = EarlyStopping(monitor='val_accuracy', patience=4)  # define early stopping callback function
    cb1 = ModelCheckpoint("/mnt/d/datasets/INBREAST/results/resnet50.h5", monitor="val_accuracy", verbose=2, save_best_only=True, mode="max")
    # cb2 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=2,
    #                         min_lr=0.00001)  # define LR reduction callback function
    opt = keras.optimizers.Adam(learning_rate=lr)
    metr = [metrics.BinaryAccuracy(name='accuracy', threshold=cutoff), metrics.AUC(name='auc'),
            metrics.Precision(name='precision'),
            metrics.Recall(name='recall')]
    #                 tfa.metrics.F1Score(name='f1_score')]
    ptmodel = ResNet50V2(include_top=False, weights='imagenet', classes=2, input_shape=(im_dim_x, im_dim_y, 3),
                         pooling='avg')  # compile resnet152v2 with imagenet weights
    ptmodel.trainable = False  # freeze layers
    ptmodel.layers[-1].trainable = True


    # un-freeze the BatchNorm layers
    # for layer in ptmodel.layers:
    #     if "BatchNormalization" in layer.__class__.__name__:
    #         layer.trainable = False # ??

    last_output = ptmodel.output
    x = tf.keras.layers.Flatten()(last_output)
    # x = tf.keras.layers.Dense(2048, activation='relu')(x)
    # #     x = tf.keras.layers.Dropout(0.15)(x)
    # x = tf.keras.layers.Dense(1024, activation='relu')(x)
    # #    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    # # x = tf.keras.layers.Dropout(0.5, seed=34)(x)
    # x = tf.keras.layers.Dense(64, activation = 'relu')(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(ptmodel.input, x)
    model.compile(optimizer=opt,
                  loss='BinaryCrossentropy',
                  metrics=metr)

2024-05-26 13:46:08.571313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 13:46:10.004062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dudder/miniconda3/envs/tf/lib/
2024-05-26 13:46:10.005652: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dudder/miniconda3/envs/tf/lib/
2024-05-26 13:46:10.005677: W tensorflow/compiler/tf2t

*************************************
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
*************************************
Found 3936 files belonging to 2 classes.


2024-05-26 13:46:18.608417: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-26 13:46:18.611700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-26 13:46:18.612187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-26 13:46:18.612529: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_nod

Found 492 files belonging to 2 classes.
Found 492 files belonging to 2 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/ta

In [2]:
print("---time taken : %s seconds ---" % (time.time() - start_time))
# Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[cb1])
print("---time taken : %s seconds ---" % (time.time() - start_time))

---time taken : 75.09799408912659 seconds ---


2024-05-26 11:43:51.444826: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 3936
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:0"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}



Epoch 1/100


2024-05-26 11:43:59.953587: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2024-05-26 11:44:00.458423: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2024-05-26 11:44:02.013183: W tensorflow/tsl/framework/bfc_allocator.cc:360] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd lik

656/656 [==============================] - ETA: 0s - loss: 0.4727 - accuracy: 0.7630 - auc: 0.8429 - precision: 0.8464 - recall: 0.7099

2024-05-26 11:44:55.891713: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_STRING
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 492
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\024TensorSliceDataset:7"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
        type_id: TFT_TENSOR
        args {
          type_id: TFT_STRING
        }
      }
    }
  }
}




Epoch 1: val_accuracy improved from -inf to 0.79065, saving model to /mnt/d/datasets/INBREAST/results/resnet50.h5
656/656 [==============================] - 128s 176ms/step - loss: 0.4727 - accuracy: 0.7630 - auc: 0.8429 - precision: 0.8464 - recall: 0.7099 - val_loss: 0.4196 - val_accuracy: 0.7907 - val_auc: 0.8815 - val_precision: 0.8492 - val_recall: 0.7670
Epoch 2/100
656/656 [==============================] - ETA: 0s - loss: 0.3846 - accuracy: 0.8079 - auc: 0.8930 - precision: 0.8853 - recall: 0.7589
Epoch 2: val_accuracy improved from 0.79065 to 0.79472, saving model to /mnt/d/datasets/INBREAST/results/resnet50.h5
656/656 [==============================] - 136s 200ms/step - loss: 0.3846 - accuracy: 0.8079 - auc: 0.8930 - precision: 0.8853 - recall: 0.7589 - val_loss: 0.3884 - val_accuracy: 0.7947 - val_auc: 0.8986 - val_precision: 0.8560 - val_recall: 0.7670
Epoch 3/100
656/656 [==============================] - ETA: 0s - loss: 0.3594 - accuracy: 0.8267 - auc: 0.9097 - precision

KeyboardInterrupt: 

In [ ]:
# Test model
# Loading checkpoint model
model.load_weights("/mnt/d/datasets/INBREAST/results/resnet50.h5")
testloss, testaccuracy, testauc, precision, recall = model.evaluate(test_ds)
print('Test accuracy :', testaccuracy)
print('Test AUC :', testauc)

F1 = 2 * float(precision) * float(recall) / (float(precision) + float(recall))
print('Test F1 :', F1)
print('Test precision :', precision)
print('Test recall :', recall)

# Model path setup.
if not os.path.exists("saved_model_resnet50_simple" + dirName):
    os.makedirs("saved_model_resnet50_simple" + dirName + '/')

model.save('saved_model_resnet50_simple' + dirName + '/resnet152v2_1')
predicted_probs = np.array([])
true_classes = np.array([])
IterationChecker = 0
for images, labels in test_ds:
    if IterationChecker == 0:
        predicted_probs = model(images)
        true_classes = labels.numpy()

    IterationChecker += 1

    predicted_probs = np.concatenate([predicted_probs,
                                      model(images)])
    true_classes = np.concatenate([true_classes, labels.numpy()])
# Since they are sigmoid outputs, you need to transform them into classes with a threshold, i.e 0.5 here:
predicted_classes = [1 * (x[0] >= cutoff) for x in predicted_probs]
# confusion matrix etc:
conf_matrix = tf.math.confusion_matrix(true_classes, predicted_classes)
print(conf_matrix)

predicted_probs = np.squeeze(predicted_probs)
predicted_classes = np.array(predicted_classes)
true_classes = np.squeeze(true_classes)
summedResults = np.stack((predicted_probs, predicted_classes, true_classes), axis=1)
##Print out statistics which test files are correctly predicted or not.
np.savetxt("Resnet50_simple_comp_EMBED.csv", summedResults, delimiter=',',
           header="predicted_probabilty,predicted_classes,true_classes", comments="")